In [1]:
!pip install "docarray[hnswlib]" transformers torch numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse

In [2]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [3]:
import pandas as pd
from docarray import BaseDoc, DocList
from docarray.typing import NdArray
from docarray.index import HnswDocumentIndex
from transformers import AutoModelForSequenceClassification, AutoModel
import torch
import numpy as np
from numpy.linalg import norm

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [4]:
# Load the MovieLens dataset
movies = pd.read_csv('ml-latest-small/movies.csv')

In [5]:
class MovieDoc(BaseDoc):
    title: str
    genres: str
    embedding: NdArray[768]  # Using Jina embeddings dimension

In [6]:
# Initialize the Jina embeddings model
embedding_model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
embedding_model.to('cuda' if torch.cuda.is_available() else 'cpu')
embedding_model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

configuration_bert.py:   0%|          | 0.00/8.24k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_bert.py:   0%|          | 0.00/97.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

JinaBertModel(
  (embeddings): JinaBertEmbeddings(
    (word_embeddings): Embedding(30528, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): JinaBertEncoder(
    (layer): ModuleList(
      (0-11): 12 x JinaBertLayer(
        (attention): JinaBertAttention(
          (self): JinaBertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): JinaBertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )

In [7]:
# Function to compute embeddings
def compute_embedding(text):
    with torch.no_grad():
        embeddings = embedding_model.encode([text])
        return embeddings[0]  # Return the first (and only) embedding

# Generate embeddings for movie titles
movies['embedding'] = movies['title'].apply(compute_embedding)


In [8]:
movies

# save dataframe to csv
#movies.to_csv('movies_with_embeddings.csv', index=False)

,movieId,title,genres,embedding
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[-0.25482577, -0.0009586513, 0.32313487, 0.612..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[-0.35054696, -0.12771174, 0.30238387, 0.30868..."
2,3,Grumpier Old Men (1995),Comedy|Romance,"[-0.49297085, -0.07011958, 0.29562932, 0.59061..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[-0.31037977, 0.2407217, 0.08884381, 0.589166,..."
4,5,Father of the Bride Part II (1995),Comedy,"[-0.27628705, -0.37252948, 0.1562669, 0.768951..."
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,"[-0.50075376, -0.15053298, 0.16756207, -0.2321..."
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"[0.18374348, -0.35666424, 0.44518903, 0.187417..."
9739,193585,Flint (2017),Drama,"[-0.23355317, 0.03771835, 0.7135002, 0.1555786..."
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"[-0.28284207, -0.61870956, 0.5151671, 0.373481..."


In [9]:
# Create and index documents
doc_index = HnswDocumentIndex[MovieDoc](work_dir='/content/data')
docs = DocList[MovieDoc](
    [MovieDoc(title=row['title'], genres=row['genres'], embedding=row['embedding']) for _, row in movies.iterrows()]
)
doc_index.index(docs)

In [10]:
# Initialize the reranker model
reranker_model = AutoModelForSequenceClassification.from_pretrained(
    'jinaai/jina-reranker-v2-base-multilingual',
    torch_dtype="auto",
    trust_remote_code=True
)
reranker_model.to('cuda' if torch.cuda.is_available() else 'cpu')
reranker_model.eval()


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

configuration_xlm_roberta.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- configuration_xlm_roberta.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_xlm_roberta.py:   0%|          | 0.00/43.8k [00:00<?, ?B/s]

xlm_padding.py:   0%|          | 0.00/9.82k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- mha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- block.py
- mlp.py
- mha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


embedding.py:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- modeling_xlm_roberta.py
- xlm_padding.py
- block.py
- embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/562M [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768)
      (token_type_embeddings): Embedding(1, 768)
    )
    (emb_drop): Dropout(p=0.1, inplace=False)
    (emb_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder): XLMRobertaEncoder(
      (layers): ModuleList(
        (0-11): 12 x Block(
          (mixer): MHA(
            (Wqkv): LinearResidual(in_features=768, out_features=2304, bias=True)
            (inner_attn): SelfAttention(
              (drop): Dropout(p=0.1, inplace=False)
            )
            (inner_cross_attn): CrossAttention(
              (drop): Dropout(p=0.1, inplace=False)
            )
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (dropout1): Dropout(p=0.1, inplace=False)
          (drop_path1): StochasticDepth(p=0.0, mode=r

In [11]:
# Function to perform search and reranking
def search_and_rerank(query, limit=20):
    # Compute query embedding
    query_embedding = compute_embedding(query)

    # Initial search
    matches, scores = doc_index.find(query_embedding, search_field='embedding', limit=limit)
    initial_results = [{"title": match.title, "genres": match.genres, "score": float(score)} for match, score in zip(matches, scores)]

    # Prepare for reranking
    documents = [result['title'] for result in initial_results]
    sentence_pairs = [[query, doc] for doc in documents]

    # Rerank
    with torch.no_grad():
        rerank_scores = reranker_model.compute_score(sentence_pairs, max_length=1024)

    # Combine results with rerank scores
    reranked_results = [
        {**result, "rerank_score": float(rerank_score)}
        for result, rerank_score in zip(initial_results, rerank_scores)
    ]

    # Sort by rerank score
    reranked_results.sort(key=lambda x: x['rerank_score'], reverse=True)

    return reranked_results

In [16]:
# Example usage
query = "apocalypse scifi zombie"
results = search_and_rerank(query)
for result in results[:5]:
    print(f"Title: {result['title']}")
    print(f"Genres: {result['genres']}")
    print(f"Initial Score: {result['score']:.4f}")
    print(f"Rerank Score: {result['rerank_score']:.4f}")
    print()


Title: Scouts Guide to the Zombie Apocalypse (2015)
Genres: Action|Comedy|Horror
Initial Score: 81.7676
Rerank Score: 0.1413

Title: Undead (2003)
Genres: Action|Comedy|Horror|Sci-Fi
Initial Score: 85.9692
Rerank Score: 0.0981

Title: Apocalypse Now (1979)
Genres: Action|Drama|War
Initial Score: 81.4158
Rerank Score: 0.0967

Title: Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979)
Genres: Horror
Initial Score: 93.2763
Rerank Score: 0.0876

Title: Zombieland (2009)
Genres: Action|Comedy|Horror
Initial Score: 84.0624
Rerank Score: 0.0716

